In [16]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [17]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Aviles,ES,2021-07-29 23:39:59,43.5547,-5.9248,65.59,88,20,4.61
1,1,Vestmannaeyjar,IS,2021-07-29 23:38:43,63.4427,-20.2734,55.67,89,85,3.06
2,2,Kenai,US,2021-07-29 23:40:48,60.5544,-151.2583,60.03,80,90,10.36
3,3,Popondetta,PG,2021-07-29 23:41:06,-8.7537,148.2534,86.58,60,69,2.28
4,4,Madimba,TZ,2021-07-29 23:41:06,-10.4000,40.3333,68.56,87,2,8.79


In [18]:
# Checking types of the dataset (gmaps only accepts int or float types)
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [19]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [29]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
cloud_layer = gmaps.heatmap_layer(locations, weights=clouds,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(cloud_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Heatmap of percent wind speed
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
wind_layer = gmaps.heatmap_layer(locations, weights=wind_speed,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(wind_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
## Vacation criteria preferences
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [34]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Popondetta,PG,2021-07-29 23:41:06,-8.7537,148.2534,86.58,60,69,2.28
7,7,Kloulklubed,PW,2021-07-29 23:41:07,7.0419,134.2556,81.10,72,99,3.76
11,11,Soto La Marina,MX,2021-07-29 23:41:08,23.7667,-98.2167,84.11,72,100,7.58
16,16,Butaritari,KI,2021-07-29 23:37:49,3.0707,172.7902,83.93,70,13,12.97
17,17,Port Blair,IN,2021-07-29 23:39:49,11.6667,92.7500,81.90,84,100,17.34
20,20,Bathsheba,BB,2021-07-29 23:35:42,13.2167,-59.5167,81.21,78,20,12.66
22,22,Hilo,US,2021-07-29 23:36:05,19.7297,-155.0900,83.93,70,20,9.22
23,23,Batticaloa,LK,2021-07-29 23:41:11,7.7102,81.6924,79.34,71,98,7.63
25,25,Jamestown,US,2021-07-29 23:38:41,42.0970,-79.2353,76.59,78,75,12.66
27,27,Nhulunbuy,AU,2021-07-29 23:41:12,-12.2333,136.7667,77.61,88,5,6.91


In [35]:
# Determine if there are null values (3 methods: count, isnull.sum(), not_null.sum())
preferred_cities_df.count()

City_ID       192
City          192
Country       192
Date          192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
dtype: int64

In [36]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Popondetta,PG,86.58,-8.7537,148.2534,
7,Kloulklubed,PW,81.10,7.0419,134.2556,
11,Soto La Marina,MX,84.11,23.7667,-98.2167,
16,Butaritari,KI,83.93,3.0707,172.7902,
17,Port Blair,IN,81.90,11.6667,92.7500,
20,Bathsheba,BB,81.21,13.2167,-59.5167,
22,Hilo,US,83.93,19.7297,-155.0900,
23,Batticaloa,LK,79.34,7.7102,81.6924,
25,Jamestown,US,76.59,42.0970,-79.2353,
27,Nhulunbuy,AU,77.61,-12.2333,136.7667,


In [37]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [42]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Popondetta,PG,86.58,-8.7537,148.2534,Timos Transit House
7,Kloulklubed,PW,81.10,7.0419,134.2556,Storyboard Beach Resort
11,Soto La Marina,MX,84.11,23.7667,-98.2167,Hotel Chicago
16,Butaritari,KI,83.93,3.0707,172.7902,Isles Sunset Lodge
17,Port Blair,IN,81.90,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
...,...,...,...,...,...,...
559,Shingu,JP,81.59,33.7333,135.9833,Shingu UI Hotel
562,Bireun,ID,79.48,5.2030,96.7009,Rumah Bp.AR. ISMAIL
563,Bacolod,PH,82.02,10.6667,122.9500,Planta Hotel & Residences
566,Victoria,HK,83.32,22.2855,114.1577,Mini Hotel Central


In [44]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [52]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Lat/dt><dd>{Lat} °</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [53]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))